In [2]:
from google.colab import files
uploaded = files.upload()


Saving WA_Fn-UseC_-Telco-Customer-Churn.csv to WA_Fn-UseC_-Telco-Customer-Churn.csv


In [3]:
import pandas as pd

df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:

df.isnull().sum()


,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [5]:
def tenure_bucket(tenure):
    if tenure <= 12:
        return '0-1 Year'
    elif tenure <= 24:
        return '1-2 Years'
    elif tenure <= 48:
        return '2-4 Years'
    else:
        return '4+ Years'

df['Tenure_Bucket'] = df['tenure'].apply(tenure_bucket)


In [6]:

service_cols = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
                'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

df['Engagement_Score'] = df[service_cols].apply(lambda row: sum(row == 'Yes'), axis=1)


In [7]:
df['Churn_Risk'] = df['Churn'].apply(lambda x: 'High' if x == 'Yes' else 'Low')


In [8]:
df[['customerID', 'Tenure_Bucket', 'Engagement_Score', 'Churn_Risk']].head()


,customerID,Tenure_Bucket,Engagement_Score,Churn_Risk
0,7590-VHVEG,0-1 Year,1,Low
1,5575-GNVDE,2-4 Years,3,Low
2,3668-QPYBK,0-1 Year,3,High
3,7795-CFOCW,2-4 Years,3,Low
4,9237-HQITU,0-1 Year,1,High


In [9]:

df.to_csv("customer360_cleaned.csv", index=False)


import os
print("Saved files:", os.listdir())


Saved files: ['.config', 'WA_Fn-UseC_-Telco-Customer-Churn.csv', 'customer360_cleaned.csv', 'sample_data']


In [10]:
from google.colab import files
files.download("customer360_cleaned.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
!pip install schedule


In [13]:
import schedule
import time

def run_etl():
    print("Running ETL pipeline...")

    df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")


    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    df.dropna(subset=['TotalCharges'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.str.strip()
    df['SeniorCitizen'] = df['SeniorCitizen'].apply(lambda x: 'Yes' if x == 1 else 'No')

    def tenure_bucket(tenure):
        if tenure <= 12:
            return '0-1 Year'
        elif tenure <= 24:
            return '1-2 Years'
        elif tenure <= 48:
            return '2-4 Years'
        else:
            return '4+ Years'
    df['Tenure_Bucket'] = df['tenure'].apply(tenure_bucket)

    service_cols = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
                    'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
    df['Engagement_Score'] = df[service_cols].apply(lambda row: sum(row == 'Yes'), axis=1)
    df['Churn_Risk'] = df['Churn'].apply(lambda x: 'High' if x == 'Yes' else 'Low')


    df.to_csv("customer360_cleaned.csv", index=False)
    print("ETL pipeline completed and data saved.")


schedule.every().day.at("09:00").do(run_etl)


for _ in range(1):
    schedule.run_pending()
    time.sleep(1)


In [15]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [17]:

output_path = '/content/drive/MyDrive/Customer360/customer360_cleaned.csv'

df.to_csv(output_path, index=False)

print("File saved to Google Drive at:", output_path)


File saved to Google Drive at: /content/drive/MyDrive/Customer360/customer360_cleaned.csv
